## Shopee Code League - Multi-Channel Contacts
## Task
For each ticket, identify all contacts from each user if they have the same contact information.

For the purpose of this question, assume that all contacts from the same Phone Number / Email are the same user.

Kaggle.com

By: Hendrik Lesmana

In [1]:
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
data = pd.read_json('contacts.json')

In [3]:
df = pd.DataFrame(data)
df = df.replace('', np.NaN)

In [4]:
%%time

em = df.groupby('Email').Id.agg(lambda x: set(x))
ph = df.groupby('Phone').Id.agg(lambda x: set(x))
oi = df.groupby('OrderId').Id.agg(lambda x: set(x))

d = {i: set() for i in df.Id}

for ids in em:
    for id in ids:
        d[id] |= set(ids)
for ids in ph:
    for id in ids:
        d[id] |= set(ids)
for ids in oi:
    for id in ids:
        d[id] |= set(ids)
        
for i in tqdm(range(6)):
    for id, ids in d.items():
        for id_ in list(ids):
            d[id] |= d[id_]

  0%|          | 0/6 [00:00<?, ?it/s]

Wall time: 44.1 s


In [5]:
id_to_contact = df.set_index('Id').Contacts.to_dict()

def get_sum_contact(ids_set):
    return sum([id_to_contact[id] for id in ids_set])

In [6]:
df['set'] = df.Id.apply(lambda x: d[x])
df['trace'] = df.set.apply(lambda x: '-'.join(map(str, sorted(list(x)))))
df['n_con'] = df.set.apply(lambda x: str(get_sum_contact(x)))
df['out'] = df.trace + ', ' + df.n_con

out = df[['Id', 'out']]
out.columns = ['ticket_id', 'ticket_trace/contacts']
out

,ticket_id,ticket_trace/contacts
0,0,"0, 1"
1,1,"1-2458-98519-115061-140081-165605-476346, 12"
2,2,"2-159312-322639-348955, 4"
3,3,"3, 0"
4,4,"4, 2"
...,...,...
499995,499995,"499995, 2"
499996,499996,"499996, 4"
499997,499997,"499997, 2"
499998,499998,"121111-499998, 5"


In [7]:
out.to_csv('submission3.csv', index=False)

In [8]:
x = pd.read_csv('submission3.csv')
x

,ticket_id,ticket_trace/contacts
0,0,"0, 1"
1,1,"1-2458-98519-115061-140081-165605-476346, 12"
2,2,"2-159312-322639-348955, 4"
3,3,"3, 0"
4,4,"4, 2"
...,...,...
499995,499995,"499995, 2"
499996,499996,"499996, 4"
499997,499997,"499997, 2"
499998,499998,"121111-499998, 5"
